# Before Setting

In [4]:
# 필요 라이브러리 다운로드
import numpy as np
import pandas as pd
import requests
import time
import os
import ssl

from bs4 import BeautifulSoup
from urllib.parse import quote
from selenium import webdriver
from collections import Counter

In [5]:
# 크롬 드라이버 생성
driver = webdriver.Chrome(executable_path = '/Users/hyeonju/Desktop/data-crawling/chromedriver')

# 크롤링 웹사이트 url 설정
url = 'https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bjBC&qvt=0&query=무료%20전시회'
print(url)

https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bjBC&qvt=0&query=무료%20전시회


In [6]:
# case 1: requests로 데이터 요청하기
# resp = requests.get(url)

# case 2: SSL 인증 제외 사이트 요청 시
resp = requests.get(url, verify=False)

# 설정한 url 로 데이터 get 함수 요청
driver.get(url)

/Users/hyeonju/opt/anaconda3/envs/mume/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search.naver.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [132]:
# # 화면상에서 스크롤 끝까지 내리기
# SCROLL_PAUSE_SEC = 1

# # 스크롤 높이 가져옴
# last_height = driver.execute_script("return document.body.scrollHeight")

# while True:
#     # 끝까지 스크롤 다운
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

#     # 1초 대기
#     time.sleep(SCROLL_PAUSE_SEC)

#     # 스크롤 다운 후 스크롤 높이 다시 가져옴
#     new_height = driver.execute_script("return document.body.scrollHeight")
#     if new_height == last_height:
#         break
#     last_height = new_height

# # click next page
# # pagination xpath: /html/body/section/div[4]/div/div/form/div[2]/div[2]/ul
#     next_page = driver.find_element_by_xpath()
#     next_page.click()
#     time.sleep(SCROLL_PAUSE_SEC)


In [8]:
# 접근한 페이지의 소스코드를 텍스트로 저장 (Str)
page_text = driver.page_source

# page_text를 html list로 변환 
html = BeautifulSoup(page_text, 'html.parser')

# print(html.prettify(formatter="html5"))

# 네이버 무료전시 url 추출 (리팩토링 필요)

In [14]:
# raw url list
# ✅ Selector: mflick > div > div > div > div > div:nth-child(1) > div.data_area > div > div.title > div > strong > a
# /html/body/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div/div[3]/div/a[2]

url=[]

# 0
for i in html.select('div.area_text_box > strong.this_text > a') :
    urls = i.attrs['href']
    url.append(urls)
driver.find_element_by_xpath('//*[@id="main_pack"]/div[2]/div[2]/div/div/div[3]/div/a[2]').click() #2

# 1
for i in html.select('div.area_text_box > strong.this_text > a') :
    urls = i.attrs['href']
    url.append(urls)
driver.find_element_by_xpath('//*[@id="main_pack"]/div[2]/div[2]/div/div/div[3]/div/a[2]').click() #2

# 12, Last Page
for i in html.select('div.area_text_box > strong.this_text > a') :
    urls = i.attrs['href']
    url.append(urls)
driver.find_element_by_xpath('//*[@id="main_pack"]/div[2]/div[2]/div/div/div[3]/div/a[2]').click() #2

print(f"전시회{len(url)}개의 주소입니다.")

print(url)

전시회52개의 주소입니다.
['?where=nexearch&sm=tab_etc&mra=bjBC&pkid=360&os=29664011&qvt=0&query=%EA%B0%A4%EB%9F%AC%EB%A6%AC%ED%83%9051%EA%B8%B0%EA%B0%90%EC%98%81%EC%A3%BC%EA%B0%9C%EC%9D%B8%EC%A0%84', '?where=nexearch&sm=tab_etc&mra=bjBC&pkid=360&os=29664009&qvt=0&query=%EA%B0%A4%EB%9F%AC%EB%A6%AC%ED%83%9051%EA%B8%B0%ED%86%A0%EC%9D%B4%EC%95%84%EC%B9%98%EA%B0%9C%EC%9D%B8%EC%A0%84', '?where=nexearch&sm=tab_etc&mra=bjBC&pkid=360&os=29664005&qvt=0&query=%EA%B0%A4%EB%9F%AC%EB%A6%AC%ED%83%9051%EA%B8%B0%EC%9D%B4%EC%9D%B4%EC%98%81%EA%B0%9C%EC%9D%B8%EC%A0%84', '?where=nexearch&sm=tab_etc&mra=bjBC&pkid=360&os=29663987&qvt=0&query=%EA%B0%A4%EB%9F%AC%EB%A6%AC%ED%83%9051%EA%B8%B0%EA%B9%80%EC%86%8C%EC%A0%95%EA%B0%9C%EC%9D%B8%EC%A0%84', '?where=nexearch&sm=tab_etc&mra=bjBC&pkid=360&os=29664011&qvt=0&query=%EA%B0%A4%EB%9F%AC%EB%A6%AC%ED%83%9051%EA%B8%B0%EA%B0%90%EC%98%81%EC%A3%BC%EA%B0%9C%EC%9D%B8%EC%A0%84', '?where=nexearch&sm=tab_etc&mra=bjBC&pkid=360&os=29664009&qvt=0&query=%EA%B0%A4%EB%9F%AC%EB%A6%AC%ED%83%9

In [136]:
# https://search.naver.com/search.naver 붙여서 url 만들기
# ✅ Selector:

free_exhibition_url = []
for i in url :
  free_exhibition_url.append(f'https://search.naver.com/search.naver{i}')

print(f"네이버 무료 전시 페이지의 {len(free_exhibition_url)}개의 주소입니다")
print(free_exhibition_url)

네이버 무료 전시 페이지의 52개의 주소입니다
['https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bjBC&pkid=360&os=29664011&qvt=0&query=%EA%B0%A4%EB%9F%AC%EB%A6%AC%ED%83%9051%EA%B8%B0%EA%B0%90%EC%98%81%EC%A3%BC%EA%B0%9C%EC%9D%B8%EC%A0%84', 'https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bjBC&pkid=360&os=29664009&qvt=0&query=%EA%B0%A4%EB%9F%AC%EB%A6%AC%ED%83%9051%EA%B8%B0%ED%86%A0%EC%9D%B4%EC%95%84%EC%B9%98%EA%B0%9C%EC%9D%B8%EC%A0%84', 'https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bjBC&pkid=360&os=29664005&qvt=0&query=%EA%B0%A4%EB%9F%AC%EB%A6%AC%ED%83%9051%EA%B8%B0%EC%9D%B4%EC%9D%B4%EC%98%81%EA%B0%9C%EC%9D%B8%EC%A0%84', 'https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bjBC&pkid=360&os=29663987&qvt=0&query=%EA%B0%A4%EB%9F%AC%EB%A6%AC%ED%83%9051%EA%B8%B0%EA%B9%80%EC%86%8C%EC%A0%95%EA%B0%9C%EC%9D%B8%EC%A0%84', 'https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bjBC&pkid=360&os=29664011&qvt=0&query=%EA%B0%A4%EB%

---
# 🛠 데이터 추출 테스트

In [137]:
# Page 1, first card exhibition data crawling

exhibition_url_1_1 = free_exhibition_url[0]

request = requests.get(exhibition_url_1_1)

driver.get(exhibition_url_1_1)

In [138]:
# 접근한 페이지의 소스코드를 텍스트로 저장 (Str)
html_text = driver.page_source

# page_text를 html list로 변환 
html = BeautifulSoup(html_text, 'html.parser')

In [139]:
# selector: 
# main_pack > div.sc_new.cs_common_module.case_normal.color_23._kgs_art_exhibition > div.cm_top_wrap._sticky._custom_select._header > div.title_area._title_area > h2 > span.area_text_title > strong > a

# 제목
title_1_1 = html.select_one('div.title_area._title_area > h2 > span.area_text_title > strong > a').text
print(title_1_1)


갤러리탐 51기 : 감영주 개인전


In [140]:
# 장소
# main_pack > div.sc_new.cs_common_module.case_normal.color_23._kgs_art_exhibition > div.cm_content_wrap > div.cm_content_area._cm_content_area_info > div > div.detail_info > dl > div:nth-child(4) > dd > a
place_1_1 = html.select_one('dl.info > div:nth-child(4) > dd > a').text
print(place_1_1)


탐앤탐스 창동본점


In [141]:
# 날짜
# main_pack > div.sc_new.cs_common_module.case_normal.color_23._kgs_art_exhibition > div.cm_content_wrap > div.cm_content_area._cm_content_area_info > div > div.detail_info > dl > div:nth-child(2) > dd
data_1_1 = html.select_one('dl.info > div:nth-child(2) > dd').text
print(f'전체 기간: {data_1_1}')

startAt_1_1 = data_1_1.split(' ~')[0]
print(f'전시 시작일 : {startAt_1_1}')

endAt_1_1 = data_1_1.split(' ~')[1]
print(f'전시 종료일 : {endAt_1_1}')

전체 기간: 2023.01.04.(수) ~ 2023.04.03.(월)
전시 시작일 : 2023.01.04.(수)
전시 종료일 :  2023.04.03.(월)


In [142]:
# 연령 제한
# 해당 정보 존재하지 않아 pass

In [143]:
# price
# 무료, 별도로 추출하지 않음

In [144]:
# 전시 주최측 제공 url
# main_pack > div.sc_new.cs_common_module.case_normal.color_23._kgs_art_exhibition > div.cm_top_wrap._sticky._custom_select._header > div.title_area._title_area > h2 > span.area_text_title > strong > a
url_1_1 = html.select_one('span.area_text_title > strong > a').attrs['href']
url_1_1


'https://shopping.naver.com/art/stores/100254411/news/5000163495'

# 자동화

### 1. 세부 메서드

### 2. 통합 메서드

In [145]:
def get_crawling_exhibition_info(url):

  print(f'url : {url}')
    
  resp = requests.get(url)
    
  print(f'resp : {resp}')
    
  driver.get (url)

  page_text = driver.page_source

  html = BeautifulSoup(page_text, 'html.parser')

  # get title
  title = html.select_one('div.title_area._title_area > h2 > span.area_text_title > strong > a').text
  
  # get place
  place = html.select_one('dl.info > div:nth-child(4) > dd > a').text
  
  # get starting date, ending date
  date = html.select_one('dl.info > div:nth-child(2) > dd').text
  startAt = date.split(' ~')[0]
  endAt = date.split(' ~')[1]
  
  # get detail information url
  detail_info_url = html.select_one('span.area_text_title > strong > a').attrs['href']

  

  # append
  title_list.append(title)
  place_list.append(place)
  startAt_list.append(startAt)
  endAt_list.append(endAt)
  detail_info_url_list.append(detail_info_url)

# 크롤링 실행

In [146]:
# title
title_list = []

# place
place_list = []

# date
startAt_list = []
endAt_list = []

# detail_info_url_list
detail_info_url_list = []

for url in free_exhibition_url :
    get_crawling_exhibition_info(url)
    time.sleep(3)

url : https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bjBC&pkid=360&os=29664011&qvt=0&query=%EA%B0%A4%EB%9F%AC%EB%A6%AC%ED%83%9051%EA%B8%B0%EA%B0%90%EC%98%81%EC%A3%BC%EA%B0%9C%EC%9D%B8%EC%A0%84
resp : <Response [200]>
url : https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bjBC&pkid=360&os=29664009&qvt=0&query=%EA%B0%A4%EB%9F%AC%EB%A6%AC%ED%83%9051%EA%B8%B0%ED%86%A0%EC%9D%B4%EC%95%84%EC%B9%98%EA%B0%9C%EC%9D%B8%EC%A0%84
resp : <Response [200]>
url : https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bjBC&pkid=360&os=29664005&qvt=0&query=%EA%B0%A4%EB%9F%AC%EB%A6%AC%ED%83%9051%EA%B8%B0%EC%9D%B4%EC%9D%B4%EC%98%81%EA%B0%9C%EC%9D%B8%EC%A0%84
resp : <Response [200]>


# 크롤링 데이터 확인

In [ ]:
# Verify title list
title_list

In [ ]:
# Verify place list
place_list

In [ ]:
# Verify start date list
startAt_list

In [ ]:
# Verify end date list 
endAt_list

In [ ]:
# Verify detail_info_url list
detail_info_url_list

# DataFrame

In [ ]:
data = {
  'title': title_list,
  'place': place_list,
  'startAt': startAt_list,
  'endAt': endAt_list,
  'detail_info_url': detail_info_url_list 
}

df = pd.DataFrame(data)

df

# Save to .csv File

In [ ]:
df.to_csv('not-paid-exhibitions.csv', index = False, encoding = 'utf-8-sig')